# HW5

### practice Support Vector Machine

- training
    - `X_train.csv`
        - image
        - 5000 * 784(28 * 28)
    - `T_train.csv`
        - label
        - 5000 * 1
- testing
    - `X_test.csv`
        - image
        - 2500 * 784(28 * 28)
    - `T_test.csv`
        - label
        - 2500 * 1

In [10]:
import numpy as np
from matplotlib import pyplot as plt

In [4]:
X_train = np.genfromtxt('X_train.csv', delimiter=',')
T_train = np.genfromtxt('T_train.csv', delimiter=',')
X_test = np.genfromtxt('X_test.csv', delimiter=',')
T_test = np.genfromtxt('T_test.csv', delimiter=',')

### different kernel functions

- linear
- polynomial
- RBF

### use C-SVC

use grid search and cross-validation

### linear + RBF kernel

### report